In [37]:
password='95463b11287'

In [35]:
!echo password | sudo -S docker-compose up -d --build

[sudo] пароль для alexey: Попробуйте ещё раз.
[sudo] пароль для alexey: 
sudo: no password was provided
sudo: 1 попытка ввода неправильного пароля


In [10]:
import time

from random import choice, randint
from typing import Callable
from uuid import uuid4

from faker import Faker
from pymongo import MongoClient



In [11]:
MONGO_HOST = "127.0.0.1"
MONGO_PORT = 27017
MONGO_DB = "ugc_db"
MONGO_COLLECTION_LIKE = "likedFilms"
MONGO_COLLECTION_REVIEW = "reviews"
MONGO_COLLECTION_BOOKMARK = "bookmarks"
ITERATIONS_NUMBER = 10
USERS_IN_BATCH = 10
OPTIMAL_BATCH_SIZE = 200
TEST_RECORDS_SIZE = 10000

LIKE = 1
DISLIKE = 0
START_DATE = "-30d"
END_DATE = "now"
MIN_RATING = 1
MAX_RATING = 10

client = MongoClient(MONGO_HOST, MONGO_PORT, connect=True)
mongo_db = client[MONGO_DB]

fake = Faker()

In [12]:

def fake_like_event(user_id: str = None, film_id: str = None) -> dict:
    """Генерация события like."""
    return {
        "user_id": user_id if user_id else str(uuid4()),
        "film_id": film_id if film_id else str(uuid4()),
        "type": choice([LIKE, DISLIKE]),
        "datetime": fake.date_time_between(start_date=START_DATE, end_date=END_DATE),
    }

In [13]:
def fake_review_event(user_id: str = None, film_id: str = None) -> dict:
    """Генерация события review."""
    return {
        "user_id": user_id if user_id else str(uuid4()),
        "film_id": film_id if film_id else str(uuid4()),
        "text": fake.text(),
        "rating": randint(MIN_RATING, MAX_RATING),
        "datetime": fake.date_time_between(start_date=START_DATE, end_date=END_DATE),
    }

In [14]:
def fake_bookmark_event(user_id: str = None, film_id: str = None) -> dict:
    """Генерация события bookmark."""
    return {
        "user_id": user_id if user_id else str(uuid4()),
        "film_id": film_id if film_id else str(uuid4()),
        "datetime": fake.date_time_between(start_date=START_DATE, end_date=END_DATE),
    }

In [15]:
def fake_batch(event_faker: Callable, user_size: int, batch_size: int) -> list[dict]:
    """Генерация батча событий."""
    users = [str(uuid4()) for _ in range(user_size)]
    return [event_faker(user_id=choice(users)) for _ in range(batch_size)]

In [16]:


def fake_users_batch(event_faker: Callable, users: list, batch_size: int) -> list[dict]:
    """Генерация батча событий с фиксированными юзерами."""
    return [event_faker(user_id=choice(users)) for _ in range(batch_size)]



In [17]:


def test_insert_step(
        faker: Callable,
        collection_name: str,
        batch_size: int,
        iterations: int = ITERATIONS_NUMBER,
) -> None:
    """Тестирование вставки."""
    collection = mongo_db.get_collection(collection_name)
    statistics = []
    for _ in range(iterations):
        batch = fake_batch(faker, USERS_IN_BATCH, batch_size)
        start = time.time()
        collection.insert_many(batch)
        end = time.time()
        statistics.append(end - start)
    mean_batch = sum(statistics) / len(statistics)
    print(
        f"Statistics for {collection_name} batch_size={batch_size}: batch={mean_batch} sec, "
        f"item={mean_batch/batch_size} sec.",
    )



In [18]:

def test_insert(faker: Callable, collection_name: str) -> None:
    """Тестирование вставки с разным размером батча."""
    batch_sizes = [1, 10, 50, 100, 200, 500, 1000, 2000, 5000]
    for batch_size in batch_sizes:
        test_insert_step(faker, collection_name, batch_size)



In [19]:
test_insert(
        fake_like_event,
        MONGO_COLLECTION_LIKE,
    )



Statistics for likedFilms batch_size=1: batch=0.004756402969360351 sec, item=0.004756402969360351 sec.
Statistics for likedFilms batch_size=10: batch=0.002495002746582031 sec, item=0.00024950027465820313 sec.
Statistics for likedFilms batch_size=50: batch=0.004588079452514648 sec, item=9.176158905029296e-05 sec.
Statistics for likedFilms batch_size=100: batch=0.004948234558105469 sec, item=4.948234558105469e-05 sec.
Statistics for likedFilms batch_size=200: batch=0.009695076942443847 sec, item=4.8475384712219234e-05 sec.
Statistics for likedFilms batch_size=500: batch=0.013422870635986328 sec, item=2.6845741271972655e-05 sec.
Statistics for likedFilms batch_size=1000: batch=0.021298599243164063 sec, item=2.1298599243164064e-05 sec.
Statistics for likedFilms batch_size=2000: batch=0.03993353843688965 sec, item=1.9966769218444825e-05 sec.
Statistics for likedFilms batch_size=5000: batch=0.10030264854431152 sec, item=2.0060529708862305e-05 sec.


In [20]:
test_insert(
        fake_review_event,
        MONGO_COLLECTION_REVIEW,
    )

Statistics for reviews batch_size=1: batch=0.005237293243408203 sec, item=0.005237293243408203 sec.
Statistics for reviews batch_size=10: batch=0.0033991098403930663 sec, item=0.00033991098403930665 sec.
Statistics for reviews batch_size=50: batch=0.0047527790069580075 sec, item=9.505558013916014e-05 sec.
Statistics for reviews batch_size=100: batch=0.005927371978759766 sec, item=5.9273719787597655e-05 sec.
Statistics for reviews batch_size=200: batch=0.00798203945159912 sec, item=3.9910197257995605e-05 sec.
Statistics for reviews batch_size=500: batch=0.014413666725158692 sec, item=2.8827333450317383e-05 sec.
Statistics for reviews batch_size=1000: batch=0.021346402168273926 sec, item=2.1346402168273925e-05 sec.
Statistics for reviews batch_size=2000: batch=0.045159554481506346 sec, item=2.2579777240753175e-05 sec.
Statistics for reviews batch_size=5000: batch=0.13674120903015136 sec, item=2.734824180603027e-05 sec.


In [21]:
test_insert(
        fake_bookmark_event,
        MONGO_COLLECTION_BOOKMARK,
    )


Statistics for bookmarks batch_size=1: batch=0.004219508171081543 sec, item=0.004219508171081543 sec.
Statistics for bookmarks batch_size=10: batch=0.0028723716735839845 sec, item=0.00028723716735839844 sec.
Statistics for bookmarks batch_size=50: batch=0.004305410385131836 sec, item=8.610820770263672e-05 sec.
Statistics for bookmarks batch_size=100: batch=0.005558681488037109 sec, item=5.558681488037109e-05 sec.
Statistics for bookmarks batch_size=200: batch=0.007251930236816406 sec, item=3.625965118408203e-05 sec.
Statistics for bookmarks batch_size=500: batch=0.011615991592407227 sec, item=2.323198318481445e-05 sec.
Statistics for bookmarks batch_size=1000: batch=0.027324652671813963 sec, item=2.7324652671813965e-05 sec.
Statistics for bookmarks batch_size=2000: batch=0.03711709976196289 sec, item=1.8558549880981444e-05 sec.
Statistics for bookmarks batch_size=5000: batch=0.09341619014739991 sec, item=1.868323802947998e-05 sec.


In [22]:


def test_read_data(faker: Callable, collection_name: str, users_size: int) -> None:
    """Тестирование чтения."""
    statistics = []
    collection = mongo_db.get_collection(collection_name)
    users = [str(uuid4()) for _ in range(users_size)]

    for i in range(0, TEST_RECORDS_SIZE, OPTIMAL_BATCH_SIZE):
        batch = fake_users_batch(faker, users, batch_size=OPTIMAL_BATCH_SIZE)
        collection.insert_many(batch)

    for user in users:
        start = time.time()
        _ = list(collection.find({"user_id": user}))
        statistics.append(time.time() - start)

    mean_batch = sum(statistics) / len(statistics)
    print(
        f"Statistics read for {collection_name} for ~{int(TEST_RECORDS_SIZE/users_size)} records: {mean_batch} sec",
    )



In [23]:
test_read_data(
    fake_like_event, 
    MONGO_COLLECTION_LIKE, 
    20)



Statistics read for likedFilms for ~500 records: 0.08538269996643066 sec


In [24]:
test_read_data(
    fake_review_event, 
    MONGO_COLLECTION_LIKE, 
    20)

Statistics read for likedFilms for ~500 records: 0.07876452207565307 sec


In [25]:
test_read_data(
    fake_bookmark_event, 
    MONGO_COLLECTION_LIKE, 
    20)

Statistics read for likedFilms for ~500 records: 0.08223080635070801 sec


In [27]:
!echo password |sudo -S docker compose down -v

[sudo] пароль для alexey: [+] Running 0/0
 ⠋ Container mongodb_container  Stopping                                   0.1s 
[+] Running 0/1
 ⠙ Container mongodb_container  Stopping                                   0.2s 
[+] Running 0/1
 ⠹ Container mongodb_container  Stopping                                   0.3s 
[+] Running 0/1
 ⠸ Container mongodb_container  Stopping                                   0.4s 
[+] Running 0/1
 ⠼ Container mongodb_container  Stopping                                   0.5s 
[+] Running 0/1
 ⠴ Container mongodb_container  Stopping                                   0.6s 
[+] Running 2/1
 ✔ Container mongodb_container             Removed                         0.6s 
 ✔ Volume research_mongodb_data_container  Removed                         0.0s 
 ⠋ Network research_default                R...                            0.0s 
[+] Running 2/3
 ✔ Container mongodb_container             Removed                         0.6s 
 ✔ Volume research_mongodb_data_cont